In [1]:
import numpy as np
import open3d as o3d
import sympy as sp
from open3d import JVisualizer
import copy

In [2]:
w11,w12,w13,w21,w22,w23,w31,w32,w33 = sp.symbols('w11,w12,w13,w21,w22,w23,w31,w32,w33')

In [3]:
x,y,z = sp.symbols('x,y,z')

In [4]:
x_p,y_p,z_p = sp.symbols('x_p,y_p,z_p')

In [5]:
t1 = w11 * x + w12 * y + w13 * z - x_p
t2 = w21 * x + w22 * y + w23 * z - y_p
t3 = w31 * x + w32 * y + w33 * z - z_p

In [6]:
d = (t1**2 + t2**2 + t3**2)**0.5
d

((w11*x + w12*y + w13*z - x_p)**2 + (w21*x + w22*y + w23*z - y_p)**2 + (w31*x + w32*y + w33*z - z_p)**2)**0.5

In [7]:
sp.diff(d,w11)

1.0*x*((w11*x + w12*y + w13*z - x_p)**2 + (w21*x + w22*y + w23*z - y_p)**2 + (w31*x + w32*y + w33*z - z_p)**2)**(-0.5)*(w11*x + w12*y + w13*z - x_p)

In [8]:
def error(W,X,X_prime):
    y = np.matmul(X,W.T) - X_prime
    return np.dot(y,y)

def slop_error(W,X,X_prime):
    w1 = W[0,:]
    w2 = W[1,:]
    w3 = W[2,:]
    r1 = np.dot(w1,X) - X_prime[0]
    r2 = np.dot(w2,X) - X_prime[1]
    r3 = np.dot(w3,X) - X_prime[2]
    w = np.row_stack((X * r1,X * r2,X * r3))
    w /= error(W,X,X_prime)
    return w

In [9]:
def trans_form(T,source_pts):
    res_pts = []
    for p in source_pts:
        res_pts.append(np.dot(p,T.T))
    return np.array(res_pts)

In [19]:
file_path = '/home/tad/Picture/pcl-pcd/bunny.pcd'

cld = o3d.io.read_point_cloud(file_path)
cld.paint_uniform_color([0,0,1.0])


pts1 = np.asarray(cld.points)

trans = np.array([[np.cos(np.pi * 0.3),-np.sin(np.pi*0.3),0],
                  [np.sin(np.pi*0.3),np.cos(np.pi * 0.3),0],
                  [0,0,1]])
print(trans)
cld1 = o3d.geometry.PointCloud()
cld1.points = o3d.utility.Vector3dVector(trans_form(trans,pts1))
cld1.paint_uniform_color([1,0,0])

[[ 0.58778525 -0.80901699  0.        ]
 [ 0.80901699  0.58778525  0.        ]
 [ 0.          0.          1.        ]]


In [18]:
source_X = np.asarray(cld.points)
tag_X = np.asarray(cld1.points)
lz = min(len(source_X),len(tag_X))

w = np.random.normal(0,1,(3,3))
lr = 0.00003
epochs = 3000
for _ in range(epochs):
    for i in range(lz):
        delta_w = slop_error(w,source_X[i,:],tag_X[i,:])
        w -= lr * delta_w

cld3 = o3d.geometry.PointCloud()
cld3.points = o3d.utility.Vector3dVector(trans_form(w,pts1))
cld3.paint_uniform_color([0,1,0])
vis = JVisualizer()
vis.add_geometry(cld)
vis.add_geometry(cld1)
vis.add_geometry(cld3)
vis.show()
print(w)

JVisualizer with 3 geometries

[[ 0.59315399 -0.8093381  -0.0117686 ]
 [ 0.81375546  0.58914268 -0.00349174]
 [ 0.04540358  0.01786538  0.98695977]]
